In [ ]:
# Import necessary libraries
import cvxpy as cp
import numpy as np
import pandas as pd
import os

In [ ]:
# Define path and list .csv files
path_to_files = '/content'
etf_files = [file for file in os.listdir(path_to_files) if file.endswith('.csv')]

# Define the periods for segmentation
periods = {
    'daily': 1,
    'weekly': 5,
    'monthly': 21,
    'quarterly': 62,
    'yearly': 252
}

# Calculate continuously compounded returns
def calculate_returns(data, period):
    data = data.sort_values('Date')
    return np.log(data['Adj Close'] / data['Adj Close'].shift(period))

# Store the ETF data
segmented_data = {}

# Loop through each file
for etf in etf_files:
    df = pd.read_csv(os.path.join(path_to_files, etf))
    ticker = etf.split('.')[0]
    segmented_data[ticker] = {}

    # Convert the date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    print(f"\nData for {ticker}:")
    print(df.head())

    # Calculate and store returns
    for period_name, period_length in periods.items():
        segmented_data[ticker][period_name] = calculate_returns(df, period_length)
        print(f"\nReturns for {period_name} for {ticker}:")
        print(segmented_data[ticker][period_name].dropna().head())

# Loop over each ETF file
for etf_file in etf_files:
    df = pd.read_csv(os.path.join(path_to_files, etf_file))

    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Store data in a dictionary
    segmented_data[etf_file] = {period_name: calculate_returns(df, period_length)
    for period_name, period_length in periods.items()}

segmented_data[etf_files[0]]['daily'].head()


Data for EFA:
                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2005-01-03  53.396667  53.500000  52.933334  53.033333  30.926054  5273700
2005-01-04  52.930000  52.950001  51.873333  52.016666  30.333185  2983800
2005-01-05  52.066666  52.650002  51.689999  51.983334  30.313742  3143700
2005-01-06  52.099998  52.183334  51.863335  51.983334  30.313742  2673300
2005-01-07  52.513332  52.513332  51.736668  51.750000  30.177681  2475300

Returns for daily for EFA:
Date
2005-01-04   -0.019357
2005-01-05   -0.000641
2005-01-06    0.000000
2005-01-07   -0.004499
2005-01-10    0.004499
Name: Adj Close, dtype: float64

Returns for weekly for EFA:
Date
2005-01-10   -0.019998
2005-01-11   -0.002567
2005-01-12    0.006073
2005-01-13   -0.002568
2005-01-14    0.006740
Name: Adj Close, dtype: float64

Returns for monthly for EFA:
Date
2005-02-02   -0.004410
2005-02-03    0.009503
2005-02-04    0.01

Date
2005-01-03         NaN
2005-01-04   -0.019357
2005-01-05   -0.000641
2005-01-06    0.000000
2005-01-07   -0.004499
Name: Adj Close, dtype: float64

In [ ]:
# Initialize dictionaries to store the results
statistics_tables = {}
correlation_matrices = {}

# Function to compute statistical measures
def compute_statistics(df):
    stats = {
        'Mean': df.mean(),
        'StdDev': df.std(),
        'Skewness': df.skew(),
        'Kurtosis': df.kurtosis(),
        '10th Percentile': df.quantile(0.10),
        '25th Percentile': df.quantile(0.25),
        '50th Percentile': df.quantile(0.50),
        '75th Percentile': df.quantile(0.75),
        '90th Percentile': df.quantile(0.90)
    }
    return pd.DataFrame(stats, index=[df.name])

# Loop over each period to calculate statistics and correlations
for period in periods.keys():
    # Combine all ETF returns for this period into a single DataFrame
    combined_returns = pd.DataFrame({etf_file.split('.')[0]:
                                     segmented_data[etf_file][period]
                                     for etf_file in etf_files})

    combined_returns.dropna(inplace=True)

    statistics_tables[period] = pd.concat([compute_statistics
     (combined_returns[col].dropna()) for col in combined_returns], axis=0)

    # Calculate and store the correlation matrix for this period
    correlation_matrices[period] = combined_returns.corr()

periods = ['daily', 'weekly', 'monthly', 'quarterly', 'yearly']


for period in periods:
    print(f"Statistics for {period.capitalize()} Period:")
    print(statistics_tables[period])
    print("\n")

    print(f"Correlation Matrix for {period.capitalize()} Period:")
    print(correlation_matrices[period])
    print("\n\n")

Statistics for Daily Period:
         Mean    StdDev  Skewness   Kurtosis  10th Percentile  \
EFA  0.000196  0.013585 -0.366604  13.424489        -0.013537   
IWB  0.000380  0.012108 -0.552941  12.179351        -0.011857   
LQD  0.000151  0.005468 -0.450755  60.963004        -0.004914   
SHY  0.000068  0.000937  0.356509   8.729824        -0.000841   
IWM  0.000300  0.015387 -0.556457   7.039964        -0.016296   
VNQ  0.000261  0.018709 -0.525548  17.878092        -0.016270   
EEM  0.000204  0.017837  0.044101  15.537025        -0.017591   

     25th Percentile  50th Percentile  75th Percentile  90th Percentile  
EFA        -0.005339         0.000646         0.006586         0.012964  
IWB        -0.004097         0.000756         0.005904         0.012051  
LQD        -0.002117         0.000368         0.002552         0.004856  
SHY        -0.000355         0.000000         0.000473         0.001063  
IWM        -0.007124         0.000982         0.008308         0.016165  
VNQ   

In [ ]:
# Define the annual target returns and calculate the periodic targets
annual_targets = [0.02, 0.04, 0.06, 0.08]

periodic_targets = {
    'daily': [target / 252 for target in annual_targets],
    'weekly': [5 * target / 252 for target in annual_targets],
    'monthly': [21 * target / 252 for target in annual_targets],
    'quarterly': [62 * target / 252 for target in annual_targets],
    'yearly': annual_targets
}

for period, targets in periodic_targets.items():
    print(f"{period.capitalize()} target returns: {targets}")

Daily target returns: [7.936507936507937e-05, 0.00015873015873015873, 0.0002380952380952381, 0.00031746031746031746]
Weekly target returns: [0.0003968253968253968, 0.0007936507936507937, 0.0011904761904761904, 0.0015873015873015873]
Monthly target returns: [0.0016666666666666666, 0.003333333333333333, 0.005, 0.006666666666666666]
Quarterly target returns: [0.004920634920634921, 0.009841269841269842, 0.01476190476190476, 0.019682539682539683]
Yearly target returns: [0.02, 0.04, 0.06, 0.08]


In [ ]:
file_names = ['EFA.csv', 'IWB.csv', 'LQD.csv', 'SHY.csv',
              'IWM.csv', 'VNQ.csv', 'EEM.csv']
file_paths = {name: f"/content/{name}" for name in file_names}

etf_data = {}
for name, path in file_paths.items():
    etf_data[name.replace('.csv', '')] = pd.read_csv(path)

# Convert dates and set date index
for etf, df in etf_data.items():
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

# Define the periods for return calculations
periods = {'daily': 1, 'weekly': 5, 'monthly': 21,
           'quarterly': 62, 'yearly': 252}

# Function to calculate returns
def calculate_returns(df, period):
    return np.log(df['Adj Close'] / df['Adj Close'].shift(period))

# Storing segmented data
segmented_data = {period: pd.DataFrame() for period in periods}

# Populate segmented_data
for period, period_length in periods.items():
    for etf, data in etf_data.items():
        segmented_data[period][etf] = calculate_returns(data, period_length).dropna()

In [ ]:
# Check data
for period, data in segmented_data.items():
    print(f"Data for {period} period:")
    print(data.head())

Data for daily period:
                 EFA       IWB       LQD       SHY       IWM       VNQ  \
Date                                                                     
2005-01-04 -0.019357 -0.011538 -0.005628 -0.002213 -0.021670 -0.015144   
2005-01-05 -0.000641 -0.002826  0.000806  0.000247 -0.020190 -0.033807   
2005-01-06  0.000000  0.002042  0.000715  0.000491  0.005289  0.007674   
2005-01-07 -0.004499 -0.001727 -0.000447  0.000124 -0.011180 -0.002240   
2005-01-10  0.004499  0.001885 -0.001701 -0.000615  0.010124 -0.003182   

                 EEM  
Date                  
2005-01-04 -0.031273  
2005-01-05 -0.012317  
2005-01-06 -0.000680  
2005-01-07  0.001934  
2005-01-10  0.001253  
Data for weekly period:
                 EFA       IWB       LQD       SHY       IWM       VNQ  \
Date                                                                     
2005-01-10 -0.019998 -0.012165 -0.006255 -0.001966 -0.037626 -0.046699   
2005-01-11 -0.002567 -0.004086  0.001701  0.000492 

In [ ]:
mean_returns = {period: data.mean(axis=0) for period,
                data in segmented_data.items()}
covariance_matrices = {period: data.cov() for period,
                       data in segmented_data.items()}

In [ ]:
def optimize_portfolio(mean_returns, covariance_matrix, target_return):
    n_assets = len(mean_returns)
    weights = cp.Variable(n_assets)

    # Objective: Minimize portfolio variance
    objective = cp.Minimize(cp.quad_form(weights, covariance_matrix))

    # Constraints
    constraints = [
        cp.sum(weights) == 1,  # Full investment
        weights >= 0,          # Non-negative allocations
        cp.matmul(weights, mean_returns) >= target_return
    ]

    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    if problem.status not in ["infeasible", "unbounded"]:
        # If problem is solvable
        return weights.value
    else:
        print(f"Optimization problem was {problem.status}")
        return None

In [ ]:
# Define annual targets and calculate their periodic equivalents
annual_targets = [0.02, 0.04, 0.06, 0.08]
period_days = {'daily': 252, 'weekly': 52,
               'monthly': 12, 'quarterly': 4, 'yearly': 1}
periodic_targets = {period: [target /
                             period_days[period] for target in annual_targets]
                    for period in period_days}

optimized_portfolios = {}

for period in periodic_targets:
    if period in mean_returns:
        optimized_portfolios[period] = {}
        for target in periodic_targets[period]:
            optimized_portfolio = optimize_portfolio(mean_returns[period],
                                                     covariance_matrices[period]
                                                     , target)
            optimized_portfolios[period][f"Target {target * period_days[period]:.2%}"] = optimized_portfolio

for period, portfolios in optimized_portfolios.items():
    print(f"\nOptimized Portfolios for {period.capitalize()}:")
    for target, weights in portfolios.items():
        if weights is not None:
            print(f"{target}: {weights}")
        else:
            print(f"{target}: Optimization failed or was infeasible.")


Optimized Portfolios for Daily:
Target 2.00%: [2.84360824e-24 3.75713548e-02 3.65513884e-24 9.62428645e-01
 9.68490813e-25 1.66591493e-24 2.68970750e-24]
Target 4.00%: [-1.28312176e-16  2.38866576e-01  1.95939227e-01  5.65194197e-01
 -1.12233628e-16 -1.20053979e-16 -1.66413177e-16]
Target 6.00%: [9.08976196e-17 4.32056419e-01 4.22140809e-01 1.45802772e-01
 7.95065067e-17 8.50464572e-17 1.17887397e-16]
Target 8.00%: [ 6.54836782e-25  7.25329379e-01  2.74670621e-01 -7.23789507e-25
  1.76205723e-24  1.43835878e-24  8.68027167e-25]

Optimized Portfolios for Weekly:
Target 2.00%: [2.78379537e-24 2.86536157e-02 3.66881686e-24 9.71346384e-01
 8.96276768e-25 1.60409671e-24 2.56595436e-24]
Target 4.00%: [-1.08967747e-17  2.44196369e-01  1.10201950e-01  6.45601681e-01
 -9.51655071e-18 -4.41829159e-18 -1.38946360e-17]
Target 6.00%: [-1.56174510e-18  4.47222434e-01  2.67292054e-01  2.85485511e-01
 -1.36389989e-18 -6.33223596e-19 -1.99136230e-18]
Target 8.00%: [1.18589220e-24 6.77425945e-01 3.2257

In [ ]:
def calculate_expected_return(weights, returns):
    """Calculate the expected return of the portfolio."""
    mean_returns = np.mean(returns, axis=0)
    return np.dot(mean_returns, weights)

def print_expected_returns(optimized_portfolios, segmented_data):
    """Calculate expected returns for each portfolio in optimized_portfolios."""
    for period, portfolios in optimized_portfolios.items():
        print(f"\nExpected Returns for {period.capitalize()} Portfolios:")
        for target, weights in portfolios.items():
            if weights is not None:
                if isinstance(segmented_data[period], pd.DataFrame):
                    period_returns = segmented_data[period].values
                else:
                    period_returns = segmented_data[period]

                exp_return = calculate_expected_return(weights, period_returns)
                print(f"  {target}: {exp_return:.4f}")
            else:
                print(f"  {target}: Calculation not available")

print_expected_returns(optimized_portfolios, segmented_data)


Expected Returns for Daily Portfolios:
  Target 2.00%: 0.0001
  Target 4.00%: 0.0002
  Target 6.00%: 0.0002
  Target 8.00%: 0.0003

Expected Returns for Weekly Portfolios:
  Target 2.00%: 0.0004
  Target 4.00%: 0.0008
  Target 6.00%: 0.0012
  Target 8.00%: 0.0015

Expected Returns for Monthly Portfolios:
  Target 2.00%: 0.0017
  Target 4.00%: 0.0033
  Target 6.00%: 0.0050
  Target 8.00%: 0.0067

Expected Returns for Quarterly Portfolios:
  Target 2.00%: 0.0050
  Target 4.00%: 0.0100
  Target 6.00%: 0.0150
  Target 8.00%: 0.0200

Expected Returns for Yearly Portfolios:
  Target 2.00%: 0.0200
  Target 4.00%: 0.0400
  Target 6.00%: 0.0600
  Target 8.00%: 0.0800


In [ ]:
def portfolio_std(weights, covariance_matrix):
    if weights is None:
        return None
    portfolio_variance = np.dot(weights, np.dot(covariance_matrix, weights))
    portfolio_std_dev = np.sqrt(portfolio_variance)
    return portfolio_std_dev


In [ ]:
portfolio_std_devs = {}

for period in optimized_portfolios:
    portfolio_std_devs[period] = {}
    for target, weights in optimized_portfolios[period].items():
        if weights is not None:
            std_dev = portfolio_std(np.array(weights),
                                    np.array(covariance_matrices[period]))
            portfolio_std_devs[period][target] = std_dev
        else:
            portfolio_std_devs[period][target] = "Optimization failed"

# Print the portfolio standard deviations
for period, std_devs in portfolio_std_devs.items():
    print(f"\nPortfolio Standard Deviations for {period.capitalize()}:")
    for target, std_dev in std_devs.items():
        if isinstance(std_dev, str):
            print(f"{target}: {std_dev}")
        else:
            print(f"{target}: {std_dev:.4f}")



Portfolio Standard Deviations for Daily:
Target 2.00%: 0.0009
Target 4.00%: 0.0032
Target 6.00%: 0.0061
Target 8.00%: 0.0092

Portfolio Standard Deviations for Weekly:
Target 2.00%: 0.0019
Target 4.00%: 0.0066
Target 6.00%: 0.0124
Target 8.00%: 0.0183

Portfolio Standard Deviations for Monthly:
Target 2.00%: 0.0040
Target 4.00%: 0.0137
Target 6.00%: 0.0261
Target 8.00%: 0.0387

Portfolio Standard Deviations for Quarterly:
Target 2.00%: 0.0080
Target 4.00%: 0.0234
Target 6.00%: 0.0445
Target 8.00%: 0.0659

Portfolio Standard Deviations for Yearly:
Target 2.00%: 0.0228
Target 4.00%: 0.0516
Target 6.00%: 0.0953
Target 8.00%: 0.1409


In [ ]:
def portfolio_mad(weights, returns_data):
    if weights is None:
        return None
    # Calculate portfolio mean return
    portfolio_mean = np.dot(weights, returns_data.mean())

    # Calculate absolute deviations from the portfolio mean
    absolute_deviations = np.abs(returns_data - portfolio_mean)

    # Compute mean absolute deviation, weighted by the portfolio weights
    mad = np.sum(weights * absolute_deviations.mean(axis=0))
    return mad


In [ ]:
portfolio_mads = {}

for period in optimized_portfolios:
    portfolio_mads[period] = {}
    for target, weights in optimized_portfolios[period].items():
        if weights is not None:
            mad = portfolio_mad(np.array(weights), segmented_data[period])
            portfolio_mads[period][target] = mad
        else:
            portfolio_mads[period][target] = "Optimization failed"

# Print the portfolio MADs
for period, mads in portfolio_mads.items():
    print(f"\nPortfolio Mean Absolute Deviations for {period.capitalize()}:")
    for target, mad in mads.items():
        if isinstance(mad, str):
            print(f"{target}: {mad}")
        else:
            print(f"{target}: {mad:.4f}")



Portfolio Mean Absolute Deviations for Daily:
Target 2.00%: 0.0009
Target 4.00%: 0.0029
Target 6.00%: 0.0049
Target 8.00%: 0.0066

Portfolio Mean Absolute Deviations for Weekly:
Target 2.00%: 0.0017
Target 4.00%: 0.0058
Target 6.00%: 0.0100
Target 8.00%: 0.0138

Portfolio Mean Absolute Deviations for Monthly:
Target 2.00%: 0.0042
Target 4.00%: 0.0126
Target 6.00%: 0.0208
Target 8.00%: 0.0284

Portfolio Mean Absolute Deviations for Quarterly:
Target 2.00%: 0.0089
Target 4.00%: 0.0244
Target 6.00%: 0.0387
Target 8.00%: 0.0502

Portfolio Mean Absolute Deviations for Yearly:
Target 2.00%: 0.0259
Target 4.00%: 0.0622
Target 6.00%: 0.0905
Target 8.00%: 0.1088


In [ ]:
def simulate_portfolio_returns(weights, mean_returns,
                               covariance_matrix, num_simulations=10000):
    """Simulates portfolio returns based on mean, covariance, and weights."""
    if weights is None:
        return None
    # Generate multivariate normal returns
    simulated_returns = np.random.multivariate_normal(mean_returns,
                                                      covariance_matrix,
                                                      num_simulations)
    # Calculate portfolio returns
    portfolio_returns = simulated_returns @ weights
    return portfolio_returns

In [ ]:
def calculate_var_cvar(portfolio_returns, confidence_level=0.9):
    """Calculates VaR and CVaR from portfolio returns."""
    if portfolio_returns is None:
        return None, None
    # Sort returns for percentile calculation
    sorted_returns = np.sort(portfolio_returns)
    # Calculate VaR
    var_index = int((1 - confidence_level) * len(sorted_returns))
    var = sorted_returns[var_index]
    # Calculate CVaR
    cvar = sorted_returns[:var_index].mean()
    return var, cvar

In [ ]:
portfolio_risk_metrics = {}

for period in optimized_portfolios:
    portfolio_risk_metrics[period] = {}
    for target, weights in optimized_portfolios[period].items():
        if weights is not None:
            # Simulate returns
            portfolio_returns = simulate_portfolio_returns(
                np.array(weights), mean_returns[period],
                covariance_matrices[period])
            # Calculate VaR and CVaR
            var, cvar = calculate_var_cvar(portfolio_returns)
            portfolio_risk_metrics[period][target] = {'VaR':
                                                      var, 'CVaR': cvar}
        else:
            portfolio_risk_metrics[period][target] = {'VaR':
                                                      'Optimization failed',
                                                      'CVaR':
                                                      'Optimization failed'}

for period, risks in portfolio_risk_metrics.items():
    print(f"\nPortfolio Risk Metrics for {period.capitalize()}:")
    for target, metrics in risks.items():
        print(f"{target}: VaR: {metrics['VaR']:.4f}, CVaR: {metrics['CVaR']:.4f}")


Portfolio Risk Metrics for Daily:
Target 2.00%: VaR: -0.0011, CVaR: -0.0015
Target 4.00%: VaR: -0.0040, CVaR: -0.0056
Target 6.00%: VaR: -0.0076, CVaR: -0.0105
Target 8.00%: VaR: -0.0116, CVaR: -0.0158

Portfolio Risk Metrics for Weekly:
Target 2.00%: VaR: -0.0021, CVaR: -0.0030
Target 4.00%: VaR: -0.0075, CVaR: -0.0106
Target 6.00%: VaR: -0.0150, CVaR: -0.0210
Target 8.00%: VaR: -0.0215, CVaR: -0.0300

Portfolio Risk Metrics for Monthly:
Target 2.00%: VaR: -0.0034, CVaR: -0.0053
Target 4.00%: VaR: -0.0138, CVaR: -0.0202
Target 6.00%: VaR: -0.0278, CVaR: -0.0403
Target 8.00%: VaR: -0.0422, CVaR: -0.0603

Portfolio Risk Metrics for Quarterly:
Target 2.00%: VaR: -0.0052, CVaR: -0.0092
Target 4.00%: VaR: -0.0198, CVaR: -0.0309
Target 6.00%: VaR: -0.0421, CVaR: -0.0628
Target 8.00%: VaR: -0.0621, CVaR: -0.0925

Portfolio Risk Metrics for Yearly:
Target 2.00%: VaR: -0.0095, CVaR: -0.0210
Target 4.00%: VaR: -0.0263, CVaR: -0.0510
Target 6.00%: VaR: -0.0589, CVaR: -0.1017
Target 8.00%: VaR: 

In [ ]:
def optimize_portfolio_mad(returns, target_return):
    n_assets = returns.shape[1]
    weights = cp.Variable(n_assets)

    if isinstance(returns, pd.DataFrame):
        returns = returns.values

    # Calculate the expected returns and reshape for matrix multiplication
    mean_returns = np.mean(returns, axis=0)

    # Objective: Minimize the mean absolute deviation
    portfolio_return = returns @ weights
    mean_portfolio_return = cp.matmul(weights, mean_returns)

    # Calculate deviations from the mean return
    deviations = cp.abs(portfolio_return - mean_portfolio_return)
    objective = cp.Minimize(cp.sum(deviations) / returns.shape[0])

    # Constraints
    constraints = [
        cp.sum(weights) == 1,  # Full investment
        weights >= 0,  # Non-negative weights
        cp.matmul(weights, mean_returns) >= target_return
    ]

    # Solve the optimization problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    # Check the status of the problem and return the results accordingly
    if problem.status not in ["infeasible", "unbounded"]:
        return weights.value
    else:
        print(f"Optimization problem was {problem.status}")
        return None

# Sample data and target returns for testing
data = segmented_data['daily']
annual_target = 0.04
daily_target = annual_target / 252

weights = optimize_portfolio_mad(data, daily_target)
if weights is not None:
    print("Optimized Weights:", weights)
else:
    print("Optimization failed or was infeasible.")


Optimized Weights: [-5.63770099e-16  2.21504930e-01  2.60760920e-01  5.17734150e-01
 -3.16920777e-16 -4.42331145e-16 -7.42885374e-16]


In [ ]:
def optimize_portfolio_mad(returns, target_return):
    n_assets = returns.shape[1]
    weights = cp.Variable(n_assets)

    if isinstance(returns, pd.DataFrame):
        returns = returns.values

    mean_returns = np.mean(returns, axis=0)

    portfolio_return = returns @ weights
    mean_portfolio_return = cp.matmul(weights, mean_returns)

    deviations = cp.abs(portfolio_return - mean_portfolio_return)
    objective = cp.Minimize(cp.sum(deviations) / returns.shape[0])

    constraints = [
        cp.sum(weights) == 1,
        weights >= 0,
        cp.matmul(weights, mean_returns) >= target_return
    ]

    problem = cp.Problem(objective, constraints)
    problem.solve()

    if problem.status not in ["infeasible", "unbounded"]:
        return weights.value
    else:
        print(f"Optimization problem was {problem.status}")
        return None

# Define the target returns and corresponding periodic targets
annual_targets = [0.02, 0.04, 0.06, 0.08]
period_days = {'daily': 252, 'weekly': 52, 'monthly': 12,
               'quarterly': 4, 'yearly': 1}

results = {}

for period, data in segmented_data.items():
    results[period] = {}
    periodic_targets = [target / period_days[period] for target in annual_targets]
    for target in periodic_targets:
        weights = optimize_portfolio_mad(data, target)
        results[period][f"Target {target * period_days[period]:.2%}"] = weights

# Print the results for each period and target
for period, portfolios in results.items():
    print(f"\nOptimized Portfolios for {period.capitalize()} (Minimizing MAD):")
    for target, weights in portfolios.items():
        if weights is not None:
            print(f"{target}: {weights}")
        else:
            print(f"{target}: Optimization failed or was infeasible.")



Optimized Portfolios for Daily (Minimizing MAD):
Target 2.00%: [-4.85845879e-17  3.75713548e-02 -7.02858183e-18  9.62428645e-01
 -2.85707039e-17 -5.54875908e-17 -5.13331641e-17]
Target 4.00%: [-5.63770099e-16  2.21504930e-01  2.60760920e-01  5.17734150e-01
 -3.16920777e-16 -4.42331145e-16 -7.42885374e-16]
Target 6.00%: [-2.42414040e-17  3.94310927e-01  5.63067927e-01  4.26211467e-02
  4.24383170e-17  8.12200202e-18 -7.73401264e-17]
Target 8.00%: [-5.44496396e-18  7.25329379e-01  2.74670621e-01  2.96170838e-16
  2.88910839e-17  1.82952542e-17 -3.15301521e-17]

Optimized Portfolios for Weekly (Minimizing MAD):
Target 2.00%: [-3.71691609e-17  2.86536157e-02 -1.05308132e-16  9.71346384e-01
 -3.29981817e-18 -1.47403262e-16 -3.12643262e-17]
Target 4.00%: [-1.42556424e-16  2.20873730e-01  1.97569749e-01  5.81556521e-01
 -1.14604648e-16 -1.27629790e-16 -1.73948060e-16]
Target 6.00%: [-3.98845820e-16  3.96527955e-01  4.57196170e-01  1.46275875e-01
 -1.26963201e-16 -2.54215293e-16 -7.09474564e-

In [ ]:
def print_expected_returns(optimized_portfolios, segmented_data):
    """Calculate and print expected returns for each portfolio in optimized_portfolios."""
    for period, portfolios in optimized_portfolios.items():
        print(f"\nExpected Returns for {period.capitalize()} Portfolios:")
        for target, weights in portfolios.items():
            if weights is not None:
                if isinstance(segmented_data[period], pd.DataFrame):
                    period_returns = segmented_data[period].values
                else:
                    period_returns = segmented_data[period]

                # Calculate the expected portfolio return
                exp_return = calculate_expected_return(weights, period_returns)
                print(f"  {target}: {exp_return:.4f}")
            else:
                print(f"  {target}: Calculation not available")

# Calculate expected return for a given portfolio
def calculate_expected_return(weights, returns):
    """Calculate the expected return of the portfolio."""
    mean_returns = np.mean(returns, axis=0)
    return np.dot(mean_returns, weights)

print_expected_returns(results, segmented_data)


Expected Returns for Daily Portfolios:
  Target 2.00%: 0.0001
  Target 4.00%: 0.0002
  Target 6.00%: 0.0002
  Target 8.00%: 0.0003

Expected Returns for Weekly Portfolios:
  Target 2.00%: 0.0004
  Target 4.00%: 0.0008
  Target 6.00%: 0.0012
  Target 8.00%: 0.0015

Expected Returns for Monthly Portfolios:
  Target 2.00%: 0.0017
  Target 4.00%: 0.0033
  Target 6.00%: 0.0050
  Target 8.00%: 0.0067

Expected Returns for Quarterly Portfolios:
  Target 2.00%: 0.0051
  Target 4.00%: 0.0100
  Target 6.00%: 0.0150
  Target 8.00%: 0.0200

Expected Returns for Yearly Portfolios:
  Target 2.00%: 0.0212
  Target 4.00%: 0.0400
  Target 6.00%: 0.0600
  Target 8.00%: 0.0800


In [ ]:
def portfolio_returns(weights, returns):
    """Calculate the portfolio returns from asset returns and weights."""
    return np.dot(returns, weights)

def calculate_std_dev(portfolio_rets):
    """Calculate standard deviation of the portfolio returns."""
    return np.std(portfolio_rets)

def calculate_mad(portfolio_rets):
    """Calculate mean absolute deviation of the portfolio returns."""
    return np.mean(np.abs(portfolio_rets - np.mean(portfolio_rets)))

def calculate_var_cvar(portfolio_rets, confidence_level=0.9):
    """Calculate VaR and CVaR from portfolio returns."""
    sorted_rets = np.sort(portfolio_rets)
    var_index = int((1 - confidence_level) * len(sorted_rets))
    var = sorted_rets[var_index]
    cvar = np.mean(sorted_rets[:var_index])
    return var, cvar

risk_measures = {}

for period, data in segmented_data.items():
    risk_measures[period] = {}
    for target, weights in results[period].items():
        if weights is not None and np.sum(weights) != 0:
            # Calculate portfolio returns for the given weights
            port_rets = portfolio_returns(weights, data.values)

            # Calculate risk measures
            std_dev = calculate_std_dev(port_rets)
            mad = calculate_mad(port_rets)
            var, cvar = calculate_var_cvar(port_rets)

            # Store the results
            risk_measures[period][target] = {
                'StdDev': std_dev,
                'MAD': mad,
                'VaR90%': var,
                'CVaR90%': cvar
            }
        else:
            risk_measures[period][target] = {
                'StdDev': None,
                'MAD': None,
                'VaR90%': None,
                'CVaR90%': None
            }

for period, targets in risk_measures.items():
    print(f"\nRisk Measures for {period.capitalize()} Portfolios:")
    for target, measures in targets.items():
        print(f"Target {target}:")
        for measure, value in measures.items():
            if value is not None:
                print(f"  {measure}: {value:.4f}")
            else:
                print(f"  {measure}: None")


Risk Measures for Daily Portfolios:
Target Target 2.00%:
  StdDev: 0.0009
  MAD: 0.0006
  VaR90%: -0.0008
  CVaR90%: -0.0015
Target Target 4.00%:
  StdDev: 0.0033
  MAD: 0.0021
  VaR90%: -0.0030
  CVaR90%: -0.0057
Target Target 6.00%:
  StdDev: 0.0061
  MAD: 0.0038
  VaR90%: -0.0054
  CVaR90%: -0.0106
Target Target 8.00%:
  StdDev: 0.0092
  MAD: 0.0058
  VaR90%: -0.0087
  CVaR90%: -0.0167

Risk Measures for Weekly Portfolios:
Target Target 2.00%:
  StdDev: 0.0019
  MAD: 0.0012
  VaR90%: -0.0014
  CVaR90%: -0.0029
Target Target 4.00%:
  StdDev: 0.0066
  MAD: 0.0043
  VaR90%: -0.0059
  CVaR90%: -0.0118
Target Target 6.00%:
  StdDev: 0.0126
  MAD: 0.0081
  VaR90%: -0.0115
  CVaR90%: -0.0228
Target Target 8.00%:
  StdDev: 0.0183
  MAD: 0.0122
  VaR90%: -0.0183
  CVaR90%: -0.0342

Risk Measures for Monthly Portfolios:
Target Target 2.00%:
  StdDev: 0.0040
  MAD: 0.0028
  VaR90%: -0.0023
  CVaR90%: -0.0055
Target Target 4.00%:
  StdDev: 0.0138
  MAD: 0.0094
  VaR90%: -0.0118
  CVaR90%: -0.0

In [ ]:
def optimize_portfolio_cvar(returns, target_return, confidence_level=0.9):
    n_assets = returns.shape[1]
    weights = cp.Variable(n_assets)
    alpha = cp.Variable()
    aux = cp.Variable(returns.shape[0])
    if isinstance(returns, pd.DataFrame):
        returns = returns.values

    portfolio_returns = returns @ weights

    # Objective: Minimize CVaR (
    objective = cp.Minimize(alpha + cp.sum(cp.pos(portfolio_returns - alpha))
    / (returns.shape[0] * (1 - confidence_level)))

    # Constraints
    constraints = [
        cp.sum(weights) == 1,  # Full investment
        weights >= 0,          # Non-negative weights
        cp.matmul(weights, np.mean(returns, axis=0)) >= target_return
    ]

    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    if problem.status not in ["infeasible", "unbounded"]:
        return weights.value
    else:
        print(f"Optimization problem was {problem.status}")
        return None

# Define the target returns and corresponding periodic targets
annual_targets = [0.02, 0.04, 0.06, 0.08]
period_days = {'daily': 252, 'weekly': 52, 'monthly': 12, 'quarterly': 4, 'yearly': 1}

cvar_results = {}

for period, data in segmented_data.items():
    cvar_results[period] = {}
    periodic_targets = [target / period_days[period] for target in annual_targets]
    for target in periodic_targets:
        weights = optimize_portfolio_cvar(data, target)
        cvar_results[period][f"Target {target * period_days[period]:.2%}"] = weights

# Print the results for each period and target
for period, portfolios in cvar_results.items():
    print(f"\nOptimized Portfolios for {period.capitalize()} (Minimizing CVaR):")
    for target, weights in portfolios.items():
        if weights is not None:
            print(f"{target}: {weights}")
        else:
            print(f"{target}: Optimization failed or was infeasible.")


Optimized Portfolios for Daily (Minimizing CVaR):
Target 2.00%: [-8.74528127e-18  3.75713548e-02 -2.03713773e-17  9.62428645e-01
  4.44974387e-17 -1.54199097e-17  1.70115733e-17]
Target 4.00%: [-4.72178160e-16  2.23674555e-01  2.52660378e-01  5.23665068e-01
 -4.14727495e-16 -4.99731212e-16 -5.18179016e-16]
Target 6.00%: [-2.24384998e-16  3.95205077e-01  5.59729512e-01  4.50654103e-02
 -1.87999458e-16 -2.66711006e-16 -2.73915339e-16]
Target 8.00%: [-2.27754435e-15  7.25329379e-01  2.74670621e-01  9.83157771e-15
 -1.72469219e-15 -1.75687845e-15 -2.35410181e-15]

Optimized Portfolios for Weekly (Minimizing CVaR):
Target 2.00%: [ 2.25747308e-17  2.86536157e-02 -5.32075967e-16  9.71346384e-01
  5.12150503e-16 -3.54906752e-16  4.97134724e-16]
Target 4.00%: [-3.36977058e-15  2.34353617e-01  1.47073405e-01  6.18572979e-01
 -3.03647600e-15 -3.39077868e-15 -3.55852656e-15]
Target 6.00%: [-9.04924308e-16  4.26758723e-01  3.43950165e-01  2.29291112e-01
 -1.09201797e-15 -2.02862886e-15 -2.42793004

In [ ]:
print_expected_returns(cvar_results, segmented_data)


Expected Returns for Daily Portfolios:
  Target 2.00%: 0.0001
  Target 4.00%: 0.0002
  Target 6.00%: 0.0002
  Target 8.00%: 0.0003

Expected Returns for Weekly Portfolios:
  Target 2.00%: 0.0004
  Target 4.00%: 0.0008
  Target 6.00%: 0.0012
  Target 8.00%: 0.0015

Expected Returns for Monthly Portfolios:
  Target 2.00%: 0.0017
  Target 4.00%: 0.0033
  Target 6.00%: 0.0050
  Target 8.00%: 0.0067

Expected Returns for Quarterly Portfolios:
  Target 2.00%: 0.0050
  Target 4.00%: 0.0100
  Target 6.00%: 0.0150
  Target 8.00%: 0.0200

Expected Returns for Yearly Portfolios:
  Target 2.00%: 0.0204
  Target 4.00%: 0.0400
  Target 6.00%: 0.0600
  Target 8.00%: 0.0800


In [ ]:
def portfolio_returns(weights, returns):
    """Calculate the portfolio returns from asset returns and weights."""
    return np.dot(returns, weights)

def calculate_std_dev(portfolio_rets):
    """Calculate standard deviation of the portfolio returns."""
    return np.std(portfolio_rets)

def calculate_mad(portfolio_rets):
    """Calculate mean absolute deviation of the portfolio returns."""
    return np.mean(np.abs(portfolio_rets - np.mean(portfolio_rets)))

def calculate_var_cvar(portfolio_rets, confidence_level=0.9):
    """Calculate VaR and CVaR from portfolio returns."""
    sorted_rets = np.sort(portfolio_rets)
    var_index = int((1 - confidence_level) * len(sorted_rets))
    var = sorted_rets[var_index]
    cvar = np.mean(sorted_rets[:var_index])
    return var, cvar

risk_measures_cvar = {}

for period, data in segmented_data.items():
    risk_measures_cvar[period] = {}
    for target, weights in cvar_results[period].items():
        if weights is not None and np.any(weights > 1e-5):
            # Calculate portfolio returns for the given weights
            port_rets = portfolio_returns(weights, data.values)

            # Calculate risk measures
            std_dev = calculate_std_dev(port_rets)
            mad = calculate_mad(port_rets)
            var, cvar = calculate_var_cvar(port_rets)

            # Store the results
            risk_measures_cvar[period][target] = {
                'StdDev': std_dev,
                'MAD': mad,
                'VaR90%': var,
                'CVaR90%': cvar
            }
        else:
            risk_measures_cvar[period][target] = {
                'StdDev': None,
                'MAD': None,
                'VaR90%': None,
                'CVaR90%': None
            }

for period, targets in risk_measures_cvar.items():
    print(f"\nRisk Metrics for {period.capitalize()} Portfolios (Minimizing CVaR):")
    for target, measures in targets.items():
        print(f"Target {target}:")
        for measure, value in measures.items():
            if value is not None:
                print(f"  {measure}: {value:.4f}")
            else:
                print(f"  {measure}: None")


Risk Metrics for Daily Portfolios (Minimizing CVaR):
Target Target 2.00%:
  StdDev: 0.0009
  MAD: 0.0006
  VaR90%: -0.0008
  CVaR90%: -0.0015
Target Target 4.00%:
  StdDev: 0.0033
  MAD: 0.0021
  VaR90%: -0.0030
  CVaR90%: -0.0057
Target Target 6.00%:
  StdDev: 0.0061
  MAD: 0.0038
  VaR90%: -0.0054
  CVaR90%: -0.0106
Target Target 8.00%:
  StdDev: 0.0092
  MAD: 0.0058
  VaR90%: -0.0087
  CVaR90%: -0.0167

Risk Metrics for Weekly Portfolios (Minimizing CVaR):
Target Target 2.00%:
  StdDev: 0.0019
  MAD: 0.0012
  VaR90%: -0.0014
  CVaR90%: -0.0029
Target Target 4.00%:
  StdDev: 0.0066
  MAD: 0.0043
  VaR90%: -0.0060
  CVaR90%: -0.0118
Target Target 6.00%:
  StdDev: 0.0124
  MAD: 0.0081
  VaR90%: -0.0117
  CVaR90%: -0.0227
Target Target 8.00%:
  StdDev: 0.0183
  MAD: 0.0122
  VaR90%: -0.0183
  CVaR90%: -0.0342

Risk Metrics for Monthly Portfolios (Minimizing CVaR):
Target Target 2.00%:
  StdDev: 0.0040
  MAD: 0.0029
  VaR90%: -0.0023
  CVaR90%: -0.0057
Target Target 4.00%:
  StdDev: 0.0